### Importing Libraries 

In [54]:
import cv2
import numpy as np

### Making the coordinates of the line

In [55]:
def coordinates(image,line_parameters):
    try:
        slope, intercept = line_parameters
    except TypeError:
        slope, intercept = 0.1,0.1
    y1=image.shape[0]
    y2=int(y1*(0.5))
    x1=int((y1-intercept)/slope)
    x2=int((y2-intercept)/slope)
    return np.array([x1,y1,x2,y2])

### Average of left line and right line

In [56]:
def average(image,lines):
    left_fit=[]
    right_fit=[]
    for line in lines:
        x1,y1,x2,y2=line.reshape(4)
        parameters=np.polyfit((x1,x2),(y1,y2),1)
        slope=parameters[0]
        intercept=parameters[1]
        if slope<0:
            left_fit.append((slope,intercept))
        else:
            right_fit.append((slope, intercept))
       
    left_fit_average=np.average(left_fit,axis=0)
    right_fit_average=np.average(right_fit,axis=0)
    left_line=coordinates(image,left_fit_average)
    right_line=coordinates(image,right_fit_average)
    return np.array([left_line,right_line])

### Canny Edge Detector

In [57]:
 def canny(image):
    gray=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    blur=cv2.GaussianBlur(gray,(5,5),0)
    canny=cv2.Canny(blur,50,150)
    return canny

### Diplay Line

In [58]:
def display_lines(image, lines):
    line_image=np.zeros_like(image)
    if lines is not None:
        for x1,y1,x2,y2 in lines:
            cv2.line(line_image,(x1,y1),(x2,y2),(0,0,255),10)
    return line_image

### Region of Interest

In [59]:
def region_of_interest(image):
    height=image.shape[0]
    polygons=np.array([[(200,height),(1100,height),(550,250)]])
    mask=np.zeros_like(image)
    cv2.fillPoly(mask,polygons,255)
    masked_image=cv2.bitwise_and(image, mask)
    return masked_image 

### Resulting Image

In [60]:
image=cv2.imread('test_image.jpg')
lane_image=np.copy(image)
canny_image=canny(lane_image)
cropped_image=region_of_interest(canny_image)
lines=cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength=180, maxLineGap=5)
averaged_lines=average(lane_image,lines)
line_image=display_lines(lane_image,averaged_lines)
combo_image=cv2.addWeighted(lane_image,0.9, line_image,1,1)

### Resulting Video

In [62]:
cap=cv2.VideoCapture('test2.mp4')
while(cap.isOpened()):
    _, frame=cap.read()
    canny_image=canny(frame)
    cropped_image=region_of_interest(canny_image)
    lines=cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)
    averaged_lines=average(frame,lines)
    line_image=display_lines(frame,averaged_lines)
    video=cv2.addWeighted(frame,0.8, line_image,1,1)
    cv2.imshow('result',video)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

### Displaying Image

In [61]:
cv2.imshow('result',combo_image) 
cv2.waitKey(0)

-1

### Saving Image

In [51]:
cv2.imwrite('Final.jpg',combo_image)

True